In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import Image

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import itertools

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import igraph as ig

In [ ]:
import squad_reboot

In [ ]:
reload(squad_reboot)

## Example from [SQUADTransformation.ipynb](https://github.com/caramirezal/SQUADBookChapter/blob/master/SQUADTransformation.ipynb)

In [ ]:
g1 = {'A': {'A': '+'}, 
      'B': {'A': '+', 'C': '-'}, 
      'C': {'A': '-', 'B': '-'}}

In [ ]:
B = squad_reboot.SquadRegulatoryNetwork(g1)

In [ ]:
initial_state = {"A":0, "B":1, "C":1}
_ = B.dynamic_simulation(initial_state=initial_state, h=50, t_max=10)

## Example from [SQUAD_implementation.ipynb](https://github.com/caramirezal/SQUADBookChapter/blob/master/SQUAD_implementation.ipynb)

In [ ]:
# error in original https://github.com/caramirezal/SQUADBookChapter/blob/master/SQUAD_implementation.ipynb??
g2 = {"A":{"A":"+", "B":"-", "X":"+"},  
      "B":{"A":"-", "Y":"+", "B":"+"}, 
      "X":{"Z":"-", "A":"+"},
      "Y":{"A":"-", "B":"+"},
      "Z":{"B":"+"}}


In [ ]:
B = squad_reboot.SquadRegulatoryNetwork(g2)

In [ ]:
initial_state = 0
events = {10:{"X":{"perturbation":.25}}, 
          20:{"Y":{"perturbation":.25}}}
_ = B.dynamic_simulation(events=events, initial_state=initial_state)

In [ ]:
events = {10:{"Y":{"perturbation":1}}, 
          20:{"X":{"perturbation":1}}}
_ = B.dynamic_simulation(events=events, initial_state=initial_state)

In [ ]:
events = {10:{"X":{"perturbation":1}}, 
          20:{"Y":{"perturbation":1}}}
_ = B.dynamic_simulation(events=events, initial_state=initial_state)

In [ ]:
events = {9:{"A":{"perturbation":0.5, "duration":1}}, 
          7:{"Y":{"perturbation":0.3, "duration":5}},
          10:{"X":{"perturbation":.25, "duration":5}, "Z":{"perturbation":0}}, 
          20:{"Y":{"perturbation":1}}
         }
_ = B.dynamic_simulation(events=events, initial_state=initial_state)

## A. thaliana network from graphml


In [ ]:
import xmltodict

In [ ]:
# load graph
g_path = "./examples/Athaliana.graphml"
g = ig.Graph.Read_GraphML(g_path)

# add edge attributes (i.e. activator or inhibitor)
with open(g_path) as f:
    d = xmltodict.parse(f.read())
    
D = {}
W = {}
N = {}

for v in d["graphml"]["graph"]["node"]:
    N[v["@id"]] = v["data"]["y:ShapeNode"]["y:NodeLabel"]["#text"]


for e in d["graphml"]["graph"]["edge"]:
    if e["data"]["y:PolyLineEdge"]['y:Arrows']['@target'] == "standard":
        D[e["@id"]] = "+"
        W[e["@id"]] = 1
    elif e["data"]["y:PolyLineEdge"]['y:Arrows']['@target'] == "white_diamond":
        D[e["@id"]] = "-"
        W[e["@id"]] = -1
    else: 
        print(e["@id"])
        
for e in g.es():
    e["type"] = D[e["id"]]
    e["weight"] = W[e["id"]]

for v in g.vs():
    v["id"] = N[v["id"]]
    
del D, W, N

In [ ]:
edge_color_dict = {"+":"green", "-":"red"}
layout = g.layout("circular")
visual_style = {}
visual_style["vertex_size"] = 20
visual_style["vertex_label"] = g.vs["id"]
visual_style["edge_color"] = [edge_color_dict[e["type"]] for e in g.es()]
visual_style["layout"] = layout
visual_style["bbox"] = (300, 300)
visual_style["margin"] = 40
visual_style["arrow_size"] = 900
visual_style["arrow_width"] = 0

ig.plot(g, **visual_style)

In [ ]:
# standard dictionary
g3 = {node["id"]:{} for node in g.vs()}
for e in g.es():
    source = g.vs()[e.source]["id"]
    target = g.vs()[e.target]["id"]
    sign = e["type"]
    g3[target][source] = sign
g3

In [ ]:
B = squad_reboot.SquadRegulatoryNetwork(g3)

In [ ]:
_ = B.dynamic_simulation(t_max=10)